<a href="https://colab.research.google.com/github/edwige3/NLP_Lab-NLP_Lab-3-n-gram-models/blob/main/Raissa_Edwige_Feuyo_Manfouo_Lab_3_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [ ]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [ ]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [ ]:

print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print(vocab.items())
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
dict_items([('<s>', 5561), ('my', 203), ('fathers', 1), ("don't", 199), ('speak', 12), ('dutch.', 1), ('</s>', 5561), ('tomorrow', 4), ('is', 735), ('the', 4669), ('election.', 3), ('he', 430), ('worked', 5), ('hard', 16), ('to', 1761), ('make', 44), ('his', 246), ('ideas', 6), ('understandable', 2), ('at', 258), ('meeting.', 12), ('tom', 1478), ('going', 104), ('stay', 26), ('in', 900), ('australia', 21), ('until', 24), ('end', 23), ('of', 980), ('october.', 1), ('john', 72), ('took', 36), ("lion's", 1), ('share', 8), ('lottery', 1), ('winnings;', 1), ('claimed', 1), ('it', 222), ('was', 507), ('idea', 10), ('buy', 25), ('ticket', 1), ('first', 70), ('place.', 7), ('some', 62), ('books', 11), ('are', 329), ('on', 356), ('table.', 22), ('and', 1103), ('mary', 788), ('said', 283), ('they', 835), ('were', 192), ('getting', 15), ('hungry.', 6), ('wants', 14), ('you', 582), ('be', 274), ('date', 

In [ ]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = [] # new list with unk value 
    ## FILL CODE
    for sentence in data: # for loop for each sentence in data 
        sentence_with_unk = [] # new list for each sentence with unk
        for word in sentence:
            if word in vocab and vocab[word]>mincount:
                sentence_with_unk.append(word)
            else:
                sentence_with_unk.append('<unk>')
        data_with_unk.append(sentence_with_unk)
    
    return data_with_unk

In [ ]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [ ]:
train_data

[['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>'],
 ['<s>', 'tomorrow', 'is', 'the', 'election.', '</s>'],
 ['<s>',
  'he',
  'worked',
  'hard',
  'to',
  'make',
  'his',
  'ideas',
  'understandable',
  'at',
  'the',
  'meeting.',
  '</s>'],
 ['<s>',
  'tom',
  'is',
  'going',
  'to',
  'stay',
  'in',
  'australia',
  'until',
  'the',
  'end',
  'of',
  '<unk>',
  '</s>'],
 ['<s>',
  'john',
  'took',
  'the',
  '<unk>',
  'share',
  'of',
  'the',
  '<unk>',
  '<unk>',
  'he',
  '<unk>',
  'it',
  'was',
  'his',
  'idea',
  'to',
  'buy',
  'the',
  '<unk>',
  'in',
  'the',
  'first',
  'place.',
  '</s>'],
 ['<s>', 'some', 'books', 'are', 'on', 'the', 'table.', '</s>'],
 ['<s>',
  'tom',
  'and',
  'mary',
  'said',
  'they',
  'were',
  'getting',
  'hungry.',
  '</s>'],
 ['<s>',
  'tom',
  'wants',
  'you',
  'to',
  'be',
  'his',
  'date',
  'for',
  'the',
  'prom.',
  '</s>'],
 ['<s>',
  'when',
  'i',
  'came',
  'home,',
  'my',
  'sister',
  'was',
  'playi

In [ ]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    proba (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)# convert the sentences to a tuple
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        for i in range(len(sentence)):
            total_number_words +=1
            for k in range(n):
                if i-k < 0:
                    break
                counts[sentence[i-k:i]][sentence[i]] +=1 
                           

    proba  = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
    ## FILL CODE
        denom =0
        for w in counts[context].keys():
            denom += counts[context][w]
        for w in counts[context].keys():
            proba[context][w] = counts[context][w]/denom 
    
    return proba

In [ ]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  2


In [ ]:
# RUN TO BUILD NGRAM MODEL

n = 5
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  5


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [ ]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE

    if context in model and w in model[context]:
        return model[context][w]
    else:
        return 0.4*get_prob(model, context[1:], w)


In [ ]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    prep(float): the preplexity of the model 
    '''
    ## FILL CODE
    perp, T = 0.0, 0
    for sentence in data:
        sentence = tuple(sentence)
        for i in range(1, len(sentence)):
            nc = min(n-1, i)
            context = sentence[i-nc:i]
            perp += -math.log(get_prob(model, context, sentence[i]))
            T += 1
    perp = math.exp(perp/T)
    return perp

In [ ]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 53.582886955857006


In [ ]:
# COMPUTE PERPLEXITY ON TRAINING SET

print("The perplexity is", perplexity(model, train_data, n=n))

The perplexity is 21.668651814592515


In [ ]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    
    if context in model:
        return model[context]
    else:
        return get_proba_distrib(model, context[1:])

In [ ]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        proba = get_proba_distrib(model, tuple(sentence))
        w = np.random.choice((list(proba.keys())), 1, p = list(proba.values()))
        sentence.append(w[0])
    return sentence

In [ ]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'they', 'like.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.